# Libraries and data

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
import pmdarima as pm
from pmdarima import model_selection

In [2]:
# Load the data
df = pd.read_csv('../nyc_data.csv', index_col=0, parse_dates=True)

In [3]:
# Rename variable
df = df.rename(columns = {'Demand':'y'})
df.head(0)

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,


In [4]:
# Extract regressors
X = df.iloc[:,1:]
X.head(0)

,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,


# Stationarity

In [8]:
#Test
from statsmodels.tsa.stattools import adfuller
pvalue = adfuller(x=df.y)[1]

#Condition to read test
if pvalue < 0.05:
    print(f"The Time Series is stationary. The p-value is {pvalue}")
else:
    print(f"The Time Series is not stationary. The p-value is {pvalue}")

The Time Series is not stationary. The p-value is 0.3767770707729128


In [10]:
# Differencing
df.y.diff().dropna()

Date
2015-01-02   -138.724112
2015-01-03    172.840266
2015-01-04   -131.864264
2015-01-05    163.120544
2015-01-06   -200.590336
                 ...    
2020-12-27   -227.008591
2020-12-28    312.136144
2020-12-29   -150.927771
2020-12-30     10.397643
2020-12-31    -73.629549
Name: y, Length: 2191, dtype: float64

In [11]:
pvalue = adfuller(x=df.y.diff().dropna())[1]

#Condition to read test
if pvalue < 0.05:
    print(f"The Time Series is stationary. The p-value is {pvalue}")
else:
    print(f"The Time Series is not stationary. The p-value is {pvalue}")

The Time Series is stationary. The p-value is 3.355773945632578e-22


# SARIMAX model

In [15]:
#Model
#hourly: 24, daily: 7, weekly: 52, monthly:12, quarterly:4
model = pm.ARIMA(order=(1,1,1),
                 seasonal_order=(1,1,1,7),
                 X=X,
                 suppress_warnings=True,
                 force_stationarity=False)

In [23]:
#Cross-validation
cv = model_selection.RollingForecastCV(h=31,
                                       step=16,
                                       initial=df.shape[0]-180)
cv_score = model_selection.cross_val_score(model, 
                                           y=df.y,
                                           scoring='mean_squared_error',
                                           cv=cv,
                                           verbose=2,
                                           error_score=100000000000000)

[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=3 ..........................................................
[CV] fold=4 ..........................................................
[CV] fold=5 ..........................................................
[CV] fold=6 ..........................................................
[CV] fold=7 ..........................................................
[CV] fold=8 ..........................................................
[CV] fold=9 ..........................................................


In [26]:
# CV Performance
error = np.sqrt(np.average(cv_score))

# Parameter Tuning

In [30]:
#Grid
param_grid = {'p': [0,1],
              'd': [0,1],
              'q': [0,1],
              'P': [0,1],
              'D': [0,1],
              'Q': [0,1]}
grid = ParameterGrid(param_grid)
len(list(grid))

64

In [31]:
#Parameter tuning
rmse = []

i=1
#Parameter loop
for params in grid:
    print(f"{i} / {len(list(grid))}")
    #model
    model = pm.ARIMA(order=(params['p'], params['d'], params['q']),
                 seasonal_order=(params['P'], params['D'], params['Q'], 7),
                 X=X,
                 suppress_warnings=True,
                 force_stationarity=False)

    #CV
    cv = model_selection.RollingForecastCV(h=31,
                                       step=16,
                                       initial=df.shape[0]-180)
    cv_score = model_selection.cross_val_score(model, 
                                           y=df.y,
                                           scoring='mean_squared_error',
                                           cv=cv,
                                           verbose=2,
                                           error_score=100000000000000)

    #error
    error = np.sqrt(np.average(cv_score))
    rmse.append(error)
    i += 1

1 / 64
[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=3 ..........................................................
[CV] fold=4 ..........................................................
[CV] fold=5 ..........................................................
[CV] fold=6 ..........................................................
[CV] fold=7 ..........................................................
[CV] fold=8 ..........................................................
[CV] fold=9 ..........................................................
2 / 64
[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=3 ...................................................

In [32]:
#Check the results
tuning_results = pd.DataFrame(grid)
tuning_results['rmse']=rmse
tuning_results

,D,P,Q,d,p,q,rmse
0,0,0,0,0,0,0,103.922477
1,0,0,0,0,0,1,104.210170
2,0,0,0,0,1,0,102.742861
3,0,0,0,0,1,1,85.623825
4,0,0,0,1,0,0,109.196397
...,...,...,...,...,...,...,...
59,1,1,1,0,1,1,61.600373
60,1,1,1,1,0,0,69.040203
61,1,1,1,1,0,1,60.409111
62,1,1,1,1,1,0,62.280310
